In [1]:
# ============================================================
# 📦 BLOCK 1 — Install Dependencies & Clone EDM2 Repository
# ============================================================

!pip install click tqdm psutil scipy pillow pandas --quiet

# Clone EDM2 repo
!git clone https://github.com/NVlabs/edm2.git
%cd edm2

# Try to install remaining deps (ignore if file missing)
!pip install -r requirements.txt --quiet || true

# Return to working directory
%cd /kaggle/working

print("EDM2 setup complete!")


Cloning into 'edm2'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 60 (delta 13), reused 10 (delta 10), pack-reused 33 (from 1)
Receiving objects: 100% (60/60), 1.27 MiB | 9.80 MiB/s, done.
Resolving deltas: 100% (24/24), done.
/kaggle/working/edm2
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
/kaggle/working
EDM2 setup complete!


In [2]:
# ============================================================
# 🧩 BLOCK 2 — Prepare FULL Test Landmark Labels + Filenames
# ============================================================

import pandas as pd
import numpy as np
import os

LANDMARK_CSV = "/kaggle/input/bbox-and-landmark-conditioning-csv-aligned-files/celeba64_landmarks_transformed.csv"
TEST_DIR     = "/kaggle/input/celeva-64x64-dataset/celeba64/test"
OUT_DIR      = "/kaggle/working/landmark-cond-test-labels"
os.makedirs(OUT_DIR, exist_ok=True)

# Load landmark CSV
df = pd.read_csv(LANDMARK_CSV)

img_col = df.columns[0]       # "image_id"
landmark_cols = df.columns[1:]  # 10 columns (x1,y1,...,x5,y5)

landmarks = df[landmark_cols].values.astype(np.float32)

# Get test images
test_files = sorted([f for f in os.listdir(TEST_DIR) if f.endswith(".jpg")])
print("Total test images:", len(test_files))

# Keep only those that exist in CSV
valid_test_files = [f for f in test_files if f in df[img_col].values]
print("Valid test images (in CSV):", len(valid_test_files))

# Build full label array in same test order
labels_full = np.array([
    landmarks[df[img_col].values.tolist().index(f)]
    for f in valid_test_files
], dtype=np.float32)

filenames_full = np.array(valid_test_files)

# Save arrays
np.save(f"{OUT_DIR}/landmark_labels_full.npy", labels_full)
np.save(f"{OUT_DIR}/filenames_full.npy", filenames_full)

print("Saved:")
print(f"{OUT_DIR}/landmark_labels_full.npy")
print(f"{OUT_DIR}/filenames_full.npy")


Total test images: 19962
Valid test images (in CSV): 19962
Saved:
/kaggle/working/landmark-cond-test-labels/landmark_labels_full.npy
/kaggle/working/landmark-cond-test-labels/filenames_full.npy


In [3]:
# ============================================================
# 🎨 BLOCK 3 — Write Landmark Conditional Generator Script
# ============================================================

generator_code = r'''
import os
import pickle
import torch
import numpy as np
import PIL.Image
import sys

# Ensure EDM2 repo is importable
sys.path.append("/kaggle/working/edm2")
from edm2.generate_images import edm_sampler

def generate_landmark_conditional(model, labels, names, outdir,
                                  steps=32, sigma_min=0.002, sigma_max=80, rho=7):

    os.makedirs(outdir, exist_ok=True)
    device = torch.device("cuda")

    print("Loading model:", model)
    with open(model, "rb") as f:
        net = pickle.load(f)["ema"].to(device).eval()

    # Load 10-dim labels and names
    labels_np = np.load(labels)         # shape (N,10)
    filenames = np.load(names)          # shape (N,)

    N = labels_np.shape[0]
    print(f"Generating {N} images...")

    bs_max = 64

    for start in range(0, N, bs_max):
        bs = min(bs_max, N - start)

        cond = torch.tensor(labels_np[start:start+bs], dtype=torch.float32, device=device)

        # noise input (same shape as EDM2 expects)
        noise = torch.randn(bs,
                            net.img_channels,
                            net.img_resolution,
                            net.img_resolution,
                            device=device)

        imgs = edm_sampler(
            net=net,
            noise=noise,
            labels=cond,
            num_steps=steps,
            sigma_min=sigma_min,
            sigma_max=sigma_max,
            rho=rho,
        )

        imgs = imgs.clamp(-1, 1)
        imgs = (imgs * 127.5 + 127.5).to(torch.uint8)
        imgs = imgs.permute(0, 2, 3, 1).cpu().numpy()

        for j in range(bs):
            outname = filenames[start + j].replace(".jpg", ".png")
            PIL.Image.fromarray(imgs[j]).save(os.path.join(outdir, outname))

    print("DONE! Saved images to", outdir)

if __name__ == "__main__":
    import argparse
    parser = argparse.ArgumentParser()

    parser.add_argument("--model", type=str, required=True)
    parser.add_argument("--labels", type=str, required=True)
    parser.add_argument("--names", type=str, required=True)
    parser.add_argument("--outdir", type=str, required=True)
    parser.add_argument("--steps", type=int, default=32)

    args = parser.parse_args()

    generate_landmark_conditional(
        model=args.model,
        labels=args.labels,
        names=args.names,
        outdir=args.outdir,
        steps=args.steps,
    )
'''

with open("gen_landmark_conditional.py", "w") as f:
    f.write(generator_code)

print("✅ gen_landmark_conditional.py written successfully!")


✅ gen_landmark_conditional.py written successfully!


In [4]:
# ============================================================
# 🚀 BLOCK 4 — Run Conditional Generation (FULL landmark test set)
# ============================================================

MODEL_PKL = "/kaggle/input/network-snapshot-0001572-0-100-landmark-karrasrho/pytorch/default/1/network-snapshot-0001572-0.100_landmark_karrasrho.pkl"

LABELS_FULL    = "/kaggle/working/landmark-cond-test-labels/landmark_labels_full.npy"
FILENAMES_FULL = "/kaggle/working/landmark-cond-test-labels/filenames_full.npy"

OUT_IMAGES = "/kaggle/working/landmark-cond-generated-full"

!python gen_landmark_conditional.py \
    --model "$MODEL_PKL" \
    --labels "$LABELS_FULL" \
    --names "$FILENAMES_FULL" \
    --outdir "$OUT_IMAGES" \
    --steps 32


Loading model: /kaggle/input/network-snapshot-0001572-0-100-landmark-karrasrho/pytorch/default/1/network-snapshot-0001572-0.100_landmark_karrasrho.pkl
Generating 19962 images...
DONE! Saved images to /kaggle/working/landmark-cond-generated-full


In [5]:
# ============================================================
# 📊 BLOCK 5 — Compute CelebA64 Reference Stats
# ============================================================

!python /kaggle/working/edm2/calculate_metrics.py ref \
    --data="/kaggle/input/celeva-64x64-dataset/celeba64/test" \
    --dest="/kaggle/working/celeba64_ref.pkl" \
    --metrics=fid \
    --batch=64 \
    --workers=2


Loading images from /kaggle/input/celeva-64x64-dataset/celeba64/test ...
[rank0]:[W1209 20:17:39.701460481 ProcessGroupNCCL.cpp:4561] [PG ID 0 PG GUID 0 Rank 0]  using GPU 0 to perform barrier as devices used by this process are currently unknown. This can potentially cause a hang if this rank to GPU mapping is incorrect. Specify device_ids in barrier() to force use of a particular device, or call init_process_group() with a device_id.
Setting up InceptionV3Detector...
Calculating feature statistics...
100%|██████████████████████████████████████| 312/312 [01:19<00:00,  3.90batch/s]


In [6]:
# # ============================================================
# # 🧩 BLOCK A — Create Smaller Test Set (20 Samples)
# # ============================================================

# import numpy as np
# import os

# LABELS_FULL = "/kaggle/working/landmark-cond-test-labels/landmark_labels_full.npy"
# FILENAMES_FULL = "/kaggle/working/landmark-cond-test-labels/filenames_full.npy"

# # Load full arrays
# labels_full = np.load(LABELS_FULL)
# filenames_full = np.load(FILENAMES_FULL)

# # Select first 20 images for quick test
# N_TEST = 20
# labels_20 = labels_full[:N_TEST]
# names_20  = filenames_full[:N_TEST]

# # Save small versions
# np.save("/kaggle/working/landmark-cond-test-labels/labels_20.npy", labels_20)
# np.save("/kaggle/working/landmark-cond-test-labels/names_20.npy", names_20)

# print("Created small test set:")
# print("labels_20.npy shape ->", labels_20.shape)
# print("names_20.npy count  ->", len(names_20))


In [7]:
# # ============================================================
# # 🚀 BLOCK B — Generate Images for 20 Test Samples
# # ============================================================

# MODEL_PKL = "/kaggle/input/network-snapshot-0001572-0-100-landmark-karrasrho/pytorch/default/1/network-snapshot-0001572-0.100_landmark_karrasrho.pkl"

# LABELS_20    = "/kaggle/working/landmark-cond-test-labels/labels_20.npy"
# NAMES_20     = "/kaggle/working/landmark-cond-test-labels/names_20.npy"
# OUT_IMAGES_20 = "/kaggle/working/landmark-cond-generated-20"

# !python gen_landmark_conditional.py \
#     --model "$MODEL_PKL" \
#     --labels "$LABELS_20" \
#     --names "$NAMES_20" \
#     --outdir "$OUT_IMAGES_20" \
#     --steps 8


In [8]:
# # OPTIONAL: Display few generated samples
# import os
# from PIL import Image
# import matplotlib.pyplot as plt

# folder = "/kaggle/working/landmark-cond-generated-20"
# files = sorted(os.listdir(folder))[:5]

# plt.figure(figsize=(12,5))
# for i,f in enumerate(files):
#     img = Image.open(os.path.join(folder, f))
#     plt.subplot(1,5,i+1)
#     plt.imshow(img)
#     plt.axis("off")
# plt.show()


In [9]:
# # ============================================================
# # 🔁 ONE-TIME — Compute CelebA64 Reference Stats (Test Set)
# # ============================================================

# !python /kaggle/working/edm2/calculate_metrics.py ref \
#     --data="/kaggle/input/celeva-64x64-dataset/celeba64/test" \
#     --dest="/kaggle/working/celeba64_ref.pkl" \
#     --metrics=fid \
#     --batch=64 \
#     --workers=2


In [10]:
# # ============================================================
# # OPTIONAL — Compute FID for 20 sample images
# # ============================================================

# !torchrun --standalone --nproc_per_node=1 \
#     /kaggle/working/edm2/calculate_metrics.py calc \
#     --images="/kaggle/working/landmark-cond-generated-20" \
#     --ref="/kaggle/working/celeba64_ref.pkl" \
#     --metrics=fid \
#     --num=20 \
#     --batch=20 \
#     --workers=1


In [11]:
# ============================================================
# 🧮 BLOCK 6 — Compute FID for Landmark Model Output
# ============================================================

NUM_FULL = len(os.listdir("/kaggle/working/landmark-cond-generated-full"))

!torchrun --standalone --nproc_per_node=2 \
    /kaggle/working/edm2/calculate_metrics.py calc \
    --images="/kaggle/working/landmark-cond-generated-full" \
    --ref="/kaggle/working/celeba64_ref.pkl" \
    --metrics=fid \
    --num=$NUM_FULL \
    --batch=64 \
    --workers=2


W1209 20:19:05.329000 101 torch/distributed/run.py:792] 
W1209 20:19:05.329000 101 torch/distributed/run.py:792] *****************************************
W1209 20:19:05.329000 101 torch/distributed/run.py:792] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1209 20:19:05.329000 101 torch/distributed/run.py:792] *****************************************
[W1209 20:19:05.852224813 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W1209 20:19:05.852923825 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W1209 20:19:07.836761898 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W1209 20:19:07.837623606 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W1209 20:19:07.839617743 socket.cpp: